<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/), Data Scientist at Mail.ru Group <br>
    All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #6 (demo)
## <center>  Exploring OLS, Lasso and Random Forest in a regression task
    
<img src='../../img/wine_quality.jpg' width=30%>

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [57]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [58]:
data = pd.read_csv('data/winequality-white.csv', sep=';')

In [59]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [61]:
data.shape

(4898, 12)

In [62]:
y = data['quality'] #target
X=data.drop(columns = ['quality'])

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [63]:
X_train_scaled = pd.DataFrame(X_train_scaled)
X_train_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10
0,-0.992730,0.583755,2.641169,1.924266,-0.000505,-0.544363,0.599730,1.341727,-0.321001,0.103009,-1.411199
1,-0.411415,0.387471,-0.844072,-1.030761,0.346034,0.207260,1.624343,-0.428859,-0.055511,-0.253133,-0.517292
2,0.169901,0.289329,1.506439,-0.930250,-0.736902,-1.758523,-1.798796,-0.743326,-0.785610,-0.520239,-0.110970
3,-0.992730,-1.182798,-0.519864,1.140279,-0.693584,0.207260,-0.937190,0.015496,-0.254629,-1.321558,0.214087
4,0.053638,1.270748,-0.033551,0.758337,-0.130458,-0.428729,0.320290,0.965733,0.077235,0.281080,-0.923613


In [64]:
X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
96,6.0,0.34,0.66,15.9,0.046,26.0,164.0,0.99790,3.14,0.50,8.8
2292,6.5,0.32,0.23,1.2,0.054,39.0,208.0,0.99272,3.18,0.46,9.9
1054,7.0,0.31,0.52,1.7,0.029,5.0,61.0,0.99180,3.07,0.43,10.4
3687,6.0,0.16,0.27,12.0,0.030,39.0,98.0,0.99402,3.15,0.34,10.8
596,6.9,0.41,0.33,10.1,0.043,28.0,152.0,0.99680,3.20,0.52,9.4


## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [65]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [66]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [67]:
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [68]:
linreg_coef = pd.DataFrame({'coef': linreg.coef_, 'coef_abs': np.abs(linreg.coef_)})
linreg_coef.index = X.columns
sorted_table = linreg_coef.sort_values(by='coef_abs', ascending = False)
sorted_table

,coef,coef_abs
density,-0.665720,0.665720
residual sugar,0.538164,0.538164
volatile acidity,-0.192260,0.192260
pH,0.150036,0.150036
alcohol,0.129533,0.129533
fixed acidity,0.097822,0.097822
sulphates,0.062053,0.062053
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
chlorides,0.008127,0.008127


In [69]:
print(f'Most influental feature is **{sorted_table.index[0]}**')

Most influental feature is **density**


## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [70]:
lasso1 = Lasso(0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [71]:
lasso1_coef = pd.DataFrame({'coef': lasso1.coef_, 'coef_abs': np.abs(lasso1.coef_)})
lasso1_coef.index = X.columns
sorted_table_lasso1 = lasso1_coef.sort_values(by='coef_abs', ascending = False)
sorted_table_lasso1

,coef,coef_abs
alcohol,0.322425,0.322425
residual sugar,0.256363,0.256363
density,-0.235492,0.235492
volatile acidity,-0.188479,0.188479
pH,0.067277,0.067277
free sulfur dioxide,0.043088,0.043088
sulphates,0.029722,0.029722
chlorides,-0.002747,0.002747
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000


In [28]:
print(f'Least influental feature is **{sorted_table_lasso1.index[-1]}**')

Least influental feature is **total sulfur dioxide**


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [72]:
alphas = np.logspace(-6, 2, 200)
alphas
lasso_cv = LassoCV(alphas = alphas, n_jobs=-1, random_state=17, cv=5)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=-1, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [89]:
alphas

array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-06, 7.66341087e-06, 8.40665289e-06,
       9.22197882e-06, 1.01163798e-05, 1.10975250e-05, 1.21738273e-05,
       1.33545156e-05, 1.46497140e-05, 1.60705282e-05, 1.76291412e-05,
       1.93389175e-05, 2.12145178e-05, 2.32720248e-05, 2.55290807e-05,
       2.80050389e-05, 3.07211300e-05, 3.37006433e-05, 3.69691271e-05,
       4.05546074e-05, 4.44878283e-05, 4.88025158e-05, 5.35356668e-05,
       5.87278661e-05, 6.44236351e-05, 7.06718127e-05, 7.75259749e-05,
       8.50448934e-05, 9.32930403e-05, 1.02341140e-04, 1.12266777e-04,
       1.23155060e-04, 1.35099352e-04, 1.48202071e-04, 1.62575567e-04,
      

In [73]:
lasso_cv.alpha_

0.0002833096101839324

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [74]:
lassoCV_coef = pd.DataFrame({'coef': lasso_cv.coef_, 'coef_abs': np.abs(lasso_cv.coef_)})
lassoCV_coef.index = X.columns
sorted_table_lassoCV = lassoCV_coef.sort_values(by='coef_abs', ascending = False)
sorted_table_lassoCV

,coef,coef_abs
density,-0.648161,0.648161
residual sugar,0.526883,0.526883
volatile acidity,-0.192049,0.192049
pH,0.146549,0.146549
alcohol,0.137115,0.137115
fixed acidity,0.093295,0.093295
sulphates,0.060939,0.060939
free sulfur dioxide,0.042698,0.042698
total sulfur dioxide,0.012969,0.012969
chlorides,0.006933,0.006933


In [75]:
print(f'Least influental feature is **{sorted_table_lassoCV.index[-1]}**')

Least influental feature is **citric acid**


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [76]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [77]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [85]:
cvs=cross_val_score(forest, X_train, y_train, scoring='neg_mean_squared_error')

In [86]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))      
print("Mean squared error (cv): %.3f" % np.mean(cvs))   
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.075
Mean squared error (cv): -0.462
Mean squared error (test): 0.422


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [49]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params, scoring='neg_mean_squared_error', n_jobs=-1, cv=5)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [87]:
locally_best_forest.best_params_

{'max_depth': 19, 'max_features': 7, 'min_samples_leaf': 1}

In [88]:
locally_best_forest.best_score_

-0.4346519750542217

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [53]:
locally_best_forest.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=None, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [54]:
print("Mean squared error (cv): %.3f" % np.mean(cross_val_score(locally_best_forest.best_estimator_, X_train_scaled, y_train, scoring='neg_mean_squared_error')))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, locally_best_forest.predict(X_holdout_scaled)))

Mean squared error (cv): -0.457
Mean squared error (test): 0.410


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [55]:
rf_importance = pd.DataFrame({'coef': locally_best_forest.best_estimator_.feature_importances_, 'coef_abs': np.abs(locally_best_forest.best_estimator_.feature_importances_)})
rf_importance.index = X.columns
rf_importance_sorted = rf_importance.sort_values(by='coef_abs', ascending = False)
rf_importance_sorted

,coef,coef_abs
alcohol,0.224432,0.224432
volatile acidity,0.119393,0.119393
free sulfur dioxide,0.116147,0.116147
pH,0.072806,0.072806
total sulfur dioxide,0.071318,0.071318
residual sugar,0.070160,0.070160
density,0.069367,0.069367
chlorides,0.067982,0.067982
fixed acidity,0.064268,0.064268
citric acid,0.062945,0.062945


In [56]:
print(f'Most influental feature is **{rf_importance_sorted.index[0]}**')

Most influental feature is **alcohol**


**Make conclusions about the perdormance of the explored 3 models in this particular prediction task.**